Primitive objects 

In [1]:
from gpkit import Model, Variable, parse_variables

In [2]:
import gpkit

maybe consider using the `@property` decorator to handle some of the class structuring

## Primitivies inhertied from Gpkit

In [3]:
class GpxObject(object):
    '''gpx object'''
    def togpkit(self):
        '''returns a gpkit representation of the object'''

In [4]:
class GpxModel(GpxObject):
    '''GPX model object
    
    Based on the gpkit model object
    
    Items
    -----
    gpmodel : gpkit.Model
        the gpkit model representation
        
    constr_to_expose : list
        list of the consraints which should be exposed to the public-facing interfaces
        
    added_contr : list
        constraints added to the model
    '''
    def __init__(self):
        Model.__init__(self)
    def setup(self):
        pass
    def pubdoc(self):
        '''Produces public-facing documentation of the model
        
        Returns
        -------
        string
            Documentation
        '''
        pass


In [5]:
class GpxVariable(Variable):
    '''Gpx implementation of a variable
    
    Items
    -----
    priv : boolean
        should the variable be hidden from a public user
    
    '''
    def __init__(self, *args):
        super(GpxVariable,self).__init__(*args)
        print('called init')
        self.priv = True
        

### Testing the new objects

In [6]:
thing = GpxVariable('t')

There is an issue with the inheritance that I cannot override the init method from the parent.

#### FIX: Inheritance issue

In [7]:
class Pet:    
    def __init__(self, name):
        print('pet init')
        self.name = name
        self.type = 'pet'
    def __repr__(self):
        return self.name + ' is a ' + self.type
        
class Dog(Pet):
    def __init__(self, name):
        print('dog init')        
        Pet.__init__(self, name)
#         super(Pet, self).__init__(self,name)
        self.type = 'dog'

In [8]:
Dog('spot')

dog init
pet init


spot is a dog

In [9]:
class TestVariable(Variable):
    def __init__(self, *args):
        print('class init')

In [10]:
TestVariable('t')

gpkit.Variable(t)

There seems to be a different constructor being called

In [11]:
help(Variable)

Help on class VectorizableVariable in module gpkit.nomials.variables:

class VectorizableVariable(Variable, ArrayVariable)
 |  A Variable outside a vectorized environment, an ArrayVariable within.
 |  
 |  Method resolution order:
 |      VectorizableVariable
 |      Variable
 |      gpkit.nomials.math.Monomial
 |      gpkit.nomials.math.Posynomial
 |      gpkit.nomials.math.Signomial
 |      gpkit.nomials.core.Nomial
 |      gpkit.nomials.data.NomialData
 |      ArrayVariable
 |      gpkit.nomials.array.NomialArray
 |      gpkit.repr_conventions.GPkitObject
 |      numpy.ndarray
 |      builtins.object
 |  
 |  Static methods defined here:
 |  
 |  __new__(cls, *args, **descr)
 |      Create and return a new object.  See help(type) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from Variable:
 |  
 |  __hash__(self)
 |      Return hash(self).
 |  
 |  __init__(self, *args, **descr)
 |      Initialize self. 

## Implementing Ipython repr methods

integrating with ipython and printing html: 
https://ipython.readthedocs.io/en/stable/config/integrating.html
surround with single underscores NOT the usual two

In [12]:
class Test:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name
    def _repr_markdown_(self):
        return '$t_{' + self.name + '}$'

In [13]:
t = GpxVariable('t','sec','time')`

SyntaxError: invalid syntax (<ipython-input-13-d64c9b5e1704>, line 1)

## Process

In [ ]:
# transferred to "primitives"

class Process(Model):
    '''process model
    
    Required Parameters
    -------------------
    t        [min]    process time
    stdev    [min]    standard deviation of process times
    
    Calculated Variables
    --------------------
    cv    [-]    coefficient of variation
        
    Cost Variables
    --------------
    capitalcost   [USD]   cell capital costs
    '''
    @parse_variables(__doc__, globals())
    def setup(self, t, stdev):
        '''
        Args
        ----
        t : Variable
            average process time
        stdev : Variable
            process time standard deviation
        constriants : list of constraints
            additional constriants
        '''
        self.t = t
        self.stdev = stdev
        return [cv >= stdev/t]
        

#### Model dev testing

In [ ]:
t = Variable('t')
stdev = Variable('\\sigma', 3)
constraints = [t>=stdev]

In [ ]:
thing = Process(t,stdev)

In [ ]:
thing = Model(t,constraints)

### Test Model

In [ ]:
class TestModel(Model):
    def setup_constriants(self):
        self.t = t = Variable('t', 's', 'time')
        tmin = Variable('t_{min}', 5, 's', 'time')
        self.constraints = [t >= tmin]
    
    def setup(self):
        self.setup_constriants()
        return self.constraints

In [ ]:
class Thing(Model):
    def setup(self):
        t = Variable('t', 's', 'time')
        return [t]

## Manufacturing Cost Model

In [ ]:

#TODO: think about what the parent should be
class MfgCost(GpxModel):
    '''manufacturing cost model
    
    Creates a manufacturing cost scenario based on a cost-optimized manufacturing scenario
    '''
    def __init__(self):
        '''setup
        
        Args
        ----
        cost_model : 
            the model describing cost
        
        
        '''
        # cost model
        # production line
        # processes
        # design
        pass
    
    def solve(self):
        '''solves the model'''
        pass

## Process Chain

In [ ]:
class ProcessChain(Model):
    '''process chain
    
    a process chain is an ordered list of processes that must be performed to make the part.
    the process chain also includes other constraints
    
    '''
    def setup(self,processes,additonal_constraints):
        '''gpx setup
        
        Args
        ----
        processes : list of Processes
            a list of the processes objects in the process chain
            
        additional_constraints : ConstraintSet
            the additional constraints which constrain the process chain
        '''
        pass

## Cell-Manufacturing Model

In [ ]:
## Transferred to "manufacturing"
# class QNACell(Model):
#     '''cell model for the Queueing Network Analyzer
    
#     Calculated Variables
#     --------------------    
#     m      [count]    Number of parallel workstations
#     c2a    [-]        Arrival coefficient of variation squared
#     c2s    [-]        Process coefficient of variation squared
#     c2d    [-]        Departure coefficient of variation squared
#     rho    [-]        Cell utilization
#     mu     [count/hr] Cell isolated production rate
#     W      [min]      Total flow time through cell
#     Wq     [min]      Expected queueing time
#     alpha  [-]        (1-rho)
#     CV     [-]        Process coefficient of variation
#     lam    [count/hr] Production rate
#     tnu    [min]      Average cycle time including efficiency
    
#     Margin Analysis Variables
#     -------------------------
#     chicv    1.0    [-]    Margin sensitivity to cv
#     nu       1.0    [-]    Margin sensitivity to cycle time
    
#     Note: Process time is the predicted champion time from the process models.
#           The cell cycle time takes into account the efficiency factor to get the average process time
#           The process time takes into account some non-value-add time
          
#     Note: The efficiency also gives the sensitivity to the calculated process times 
#           (which wouldn't normally have a sensitivity)
    
#     '''
#     @parse_variables(__doc__,globals())
#     def setup(self, t, cvp, nu=1):
#         '''setup the equations
        
#         Args
#         ----
#         t : variable, constant, monomial
#             the expected (average) process time
#         cvp : variable, constant, monomial
#             process time coefficient of variation
#         nu : variable, constant, monomial
#             Default=1
#             the "efficiency" of the process from champion time to expected cycle time
#         '''
        
#         self.nu = nu = Variable('\\nu_t',nu,'-','process time efficiency')
        
#         self.t = t
#         self.cvp = cvp
#         self.constraints = [
#             W >= Wq + tnu,                        # Total flow time
#             Wq >= (rho/alpha)*(c2a+c2s)/2*tnu/m,  # Kingmann approximation
#             tnu >= t*nu,                          # Process efficiency
#             c2d >= alpha*c2a + rho*c2s,
#             1 >= alpha + rho,
#             lam <= rho*m/tnu,
#             c2s >= cvp**2*chicv**2,
#         ]
#         return self.constraints
        

### Cell-based Conwip Line

In [ ]:
## Transferred to "manufacturing"

# import numpy as np
# class FabLine(Model):
#     '''
#     A CONWIP line approximated using the Queueing Network Analyzer
            
#     '''
#     def setup(self, cells):
#         '''gpx setup
        
#         Args
#         ----
#         cells : dict
#             the primary flow cells
#         '''
#         self.cells = cells
        
#         lam = self.lam = Variable('\\lambda', 'count/hr', 'Line Continuous Production Rate')
#         W   = self.W   = Variable('W', 'hr', 'Total flow time')
#         L   = self.L   = Variable('L', 'count', 'Total WIP count')
        
#         'Littles Law on Entire Line'
#         constraints = [
#             L >= lam*W,
#         ]
        
#         'Connect Variations'
#         for i,j in enumerate(cells.values()):
#             if i > 0 :
#                 constraints.append(
#                     cells.values()[i].c2a == cells.values()[i-1].c2d
#                 )
#         # Loop first and last cells
#         constraints.append(cells.values()[-1].c2d == cells.values()[0].c2a)
        
#         'Production rate has to be same everywhere'
#         constraints.extend([lam == cell.lam for cell in cells.values()])
        
#         'Find total flow time'
#         constraints.append(
#             W >= np.array([cell.W for cell in cells.values()]).sum()
#         )
#         return [constraints, self.cells.values()]

### Secondary Cell-Based Flows

In [ ]:
# transferred to "manufacturing"
# class CellSecondaryFlow(Model):
#     '''cell secondary flow models
    
#     Adds additional flows 
#     '''
#     def setup(self, cells, flows):
#         '''gpx setup
        
#         Args
#         ----
#         cells : OrderedDict
#             the cells in the primary flow
#         flows : list of tuples
#             describes the secondary flows as (arrival cell, departure cell)
            
#         '''

### Flow Manufacturing Model

## Capital Financial Model

In [ ]:
def CellCapitalFinance(Model):
    '''financial model for capital costs
    
    Calculates capital costs based on the 
    '''